<a href="https://colab.research.google.com/github/wiamfa/Speech_emotion_recognition/blob/main/huggingFace_wav2vec2-large-robust-ft-libri-960h.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
model_checkpoint = "facebook/wav2vec2-large-robust-ft-libri-960h"
batch_size = 8
ravdess_emotions = ["neutral", "calm", "happy", "sad", "angry", "fearful", "disgust", "surprised"]



In [2]:
%%capture
!pip install datasets
!pip install transformers
!pip install librosa

In [3]:
from huggingface_hub import notebook_login

notebook_login()

In [4]:
%%capture
!apt install git-lfs

In [6]:
from transformers.utils import send_example_telemetry

send_example_telemetry("audio_classification_notebook", framework="pytorch")

#test HuggingFace on Hubert

In [7]:
from datasets import load_dataset, load_metric, Audio

In [8]:
dataset = load_dataset("xbgoose/ravdess",split="train")
metric = load_metric("accuracy")

<ipython-input-8-0b02b7a0bda2>:2: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [9]:
dataset000=dataset.remove_columns([ 'modality', 'vocal_channel', 'emotional_intensity', 'statement', 'repetition', 'actor', 'gender'])

In [10]:
from datasets.tasks.text_classification import ClassLabel
#resampling to 16Khz
dataset00 = dataset000.cast_column("audio", Audio(sampling_rate=16000))
#Label identification
dataset0 = dataset00.cast_column("emotion", ClassLabel(num_classes=8, names=ravdess_emotions))


In [11]:
dataset1=dataset0.train_test_split(test_size=0.2)

In [12]:
dataset1

DatasetDict({
    train: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['audio', 'emotion'],
        num_rows: 288
    })
})

In [13]:
dataset1["test"][100]

{'audio': {'path': '03-01-02-01-02-01-09.wav',
  'array': array([-2.57571742e-14, -9.76996262e-14, -9.94759830e-14, ...,
          3.42333224e-04,  4.07578249e-04,  0.00000000e+00]),
  'sampling_rate': 16000},
 'emotion': 1}

In [14]:
dataset1["train"].features

{'audio': Audio(sampling_rate=16000, mono=True, decode=True, id=None),
 'emotion': ClassLabel(names=['neutral', 'calm', 'happy', 'sad', 'angry', 'fearful', 'disgust', 'surprised'], id=None)}

In [15]:
labels = dataset1["train"].features["emotion"].names
label2id, id2label = dict(), dict()
for i, label in enumerate(labels):
    label2id[label] = str(i)
    id2label[str(i)] = label

id2label["7"]

'surprised'

In [16]:
import random
from IPython.display import Audio, display

for _ in range(5):
    rand_idx = random.randint(0, len(dataset1["train"])-1)
    example = dataset1["train"][rand_idx]
    audio = example["audio"]

    print(f'Label: {id2label[str(example["emotion"])]}')
    print(f'Shape: {audio["array"].shape}, sampling rate: {audio["sampling_rate"]}')
    display(Audio(audio["array"], rate=audio["sampling_rate"]))
    print()

Label: angry
Shape: (65132,), sampling rate: 16000



Label: sad
Shape: (53387,), sampling rate: 16000



Label: angry
Shape: (60327,), sampling rate: 16000



Label: disgust
Shape: (67268,), sampling rate: 16000



Label: fearful
Shape: (50184,), sampling rate: 16000


In [17]:
from transformers import AutoFeatureExtractor

feature_extractor = AutoFeatureExtractor.from_pretrained(model_checkpoint)
feature_extractor

Wav2Vec2FeatureExtractor {
  "do_normalize": true,
  "feature_extractor_type": "Wav2Vec2FeatureExtractor",
  "feature_size": 1,
  "padding_side": "right",
  "padding_value": 0,
  "return_attention_mask": true,
  "sampling_rate": 16000
}

In [18]:
max_duration = 2.0  # seconds

In [19]:
def preprocess_function(examples):
    audio_arrays = [x["array"] for x in examples["audio"]]
    inputs = feature_extractor(
        audio_arrays,
        sampling_rate=feature_extractor.sampling_rate,
        max_length=int(feature_extractor.sampling_rate * max_duration),
        truncation=True,
    )
    return inputs

In [20]:
preprocess_function(dataset1['train'][:5])

{'input_values': [array([ 2.5473626e-06,  2.5473626e-06,  2.5473626e-06, ...,
       -6.9170433e-01, -4.4618174e-01, -4.0743086e-01], dtype=float32), array([-9.02689499e-05, -9.02689499e-05, -9.02689499e-05, ...,
       -1.10091425e-01, -1.11874282e-01, -1.23509243e-01], dtype=float32), array([8.7170757e-04, 3.8284943e-03, 5.6971298e-03, ..., 2.5465963e+00,
       3.1922495e+00, 3.9410264e+00], dtype=float32), array([-5.8270241e-05, -6.8556321e-05, -5.8261245e-05, ...,
        7.3829383e-02,  7.0882484e-02,  6.5752834e-02], dtype=float32), array([8.8534298e-06, 8.8534298e-06, 8.8534298e-06, ..., 2.1943009e+00,
       2.4300861e+00, 2.2033498e+00], dtype=float32)], 'attention_mask': [array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32), array([1, 1, 1, ..., 1, 1, 1], dtype=int32)]}

In [21]:
encoded_dataset = dataset1.map(preprocess_function, remove_columns=["audio"], batched=True)
encoded_dataset

Map:   0%|          | 0/1152 [00:00<?, ? examples/s]

Map:   0%|          | 0/288 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['emotion', 'input_values', 'attention_mask'],
        num_rows: 1152
    })
    test: Dataset({
        features: ['emotion', 'input_values', 'attention_mask'],
        num_rows: 288
    })
})

In [22]:
encoded_dataset["train"][0]["emotion"]

2

In [23]:
from transformers import AutoModelForAudioClassification, TrainingArguments, Trainer

num_labels = len(id2label)
model = AutoModelForAudioClassification.from_pretrained(
    model_checkpoint,
    num_labels=num_labels,
    label2id=label2id,
    id2label=id2label,
)

Some weights of Wav2Vec2ForSequenceClassification were not initialized from the model checkpoint at facebook/wav2vec2-large-robust-ft-libri-960h and are newly initialized: ['classifier.weight', 'classifier.bias', 'projector.weight', 'projector.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
!pip uninstall accelerate

Found existing installation: accelerate 0.21.0
Uninstalling accelerate-0.21.0:
  Would remove:
    /usr/local/bin/accelerate
    /usr/local/bin/accelerate-config
    /usr/local/bin/accelerate-launch
    /usr/local/lib/python3.10/dist-packages/accelerate-0.21.0.dist-info/*
    /usr/local/lib/python3.10/dist-packages/accelerate/*
Proceed (Y/n)? Y
  Successfully uninstalled accelerate-0.21.0


In [25]:
!pip install accelerate -U

  Using cached accelerate-0.21.0-py3-none-any.whl (244 kB)


In [26]:
!accelerate config



2023-08-18 14:04:19.533516: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT
In which compute environment are you running?
Please input a choice index (starting from 0), and press enter
 ➔  This machine
    AWS (Amazon SageMaker)

This machine
Which type of machine are you using?
Please input a choice index (starting from 0), and press enter
 ➔  No distributed training
    multi-CPU
    multi-XPU
    multi-GPU
    multi-NPU
    TPU

No distributed training
Do you want to run your training on CPU only (even if a GPU / Apple Silicon device is available)? [yes/NO]:NO
Do you wish to optimize your script with torch dynamo?[yes/NO]:
Do you want to use DeepSpeed? [yes/NO]: 
What GPU(s) (by id) should be used for training on this machine as a comma-seperated list? [all]:
Do you wish to use FP16 or BF16 (mixed precision)?
Please input a choice index (starting from 0), and press enter
 ➔  no
    fp16
    bf16
    fp8
1
fp16
accelerate configuration 

In [27]:
!accelerate env

2023-08-18 14:04:41.670656: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT

Copy-and-paste the text below in your GitHub issue

- `Accelerate` version: 0.21.0
- Platform: Linux-5.15.109+-x86_64-with-glibc2.35
- Python version: 3.10.12
- Numpy version: 1.23.5
- PyTorch version (GPU?): 2.0.1+cu118 (True)
- PyTorch XPU available: False
- PyTorch NPU available: False
- System RAM: 25.45 GB
- GPU type: Tesla T4
- `Accelerate` default config:
	- compute_environment: LOCAL_MACHINE
	- distributed_type: NO
	- mixed_precision: fp16
	- use_cpu: False
	- num_processes: 1
	- machine_rank: 0
	- num_machines: 1
	- gpu_ids: all
	- rdzv_backend: static
	- same_network: True
	- main_training_function: main
	- downcast_bf16: no
	- tpu_use_cluster: False
	- tpu_use_sudo: False
	- tpu_env: []


In [ ]:
model_name = model_checkpoint.split("/")[-1]

args = TrainingArguments(
    f"{model_name}-finetuned-ravdess-v2",
    evaluation_strategy = "epoch",
    save_strategy = "epoch",
    learning_rate=5e-5,
    per_device_train_batch_size=batch_size,
    gradient_accumulation_steps=4,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=20,
    warmup_ratio=0.1,
    logging_steps=10,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    push_to_hub=True,
    gradient_checkpointing=True,


)

In [29]:
import numpy as np

def compute_metrics(eval_pred):
    """Computes accuracy on a batch of predictions"""
    predictions = np.argmax(eval_pred.predictions, axis=1)
    return metric.compute(predictions=predictions, references=eval_pred.label_ids)

In [30]:
encoded_dataset["train"]

Dataset({
    features: ['emotion', 'input_values', 'attention_mask'],
    num_rows: 1152
})

In [31]:
encoded_dataset1= encoded_dataset.rename_column("emotion", "labels")

In [32]:
trainer = Trainer(
    model,
    args,
    train_dataset=encoded_dataset1["train"],
    eval_dataset=encoded_dataset1["test"],
    tokenizer=feature_extractor,
    compute_metrics=compute_metrics,
)

Cloning https://huggingface.co/Wiam/wav2vec2-large-robust-ft-libri-960h-finetuned-ravdess-v2 into local empty directory.


Download file pytorch_model.bin:   0%|          | 15.4k/1.18G [00:00<?, ?B/s]

Download file runs/Aug18_12-05-31_d6d3d67ca983/events.out.tfevents.1692360348.d6d3d67ca983.4057.0:  46%|####5 …

Download file training_args.bin: 100%|##########| 3.93k/3.93k [00:00<?, ?B/s]

Clean file training_args.bin:  25%|##5       | 1.00k/3.93k [00:00<?, ?B/s]

Clean file runs/Aug18_12-05-31_d6d3d67ca983/events.out.tfevents.1692360348.d6d3d67ca983.4057.0:   6%|5        …

Clean file pytorch_model.bin:   0%|          | 1.00k/1.18G [00:00<?, ?B/s]

In [33]:
trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss,Accuracy
1,2.078600,2.069249,0.159722
2,2.057800,2.055505,0.197917


Epoch,Training Loss,Validation Loss,Accuracy
1,2.078600,2.069249,0.159722
2,2.057800,2.055505,0.197917
3,1.990300,1.917249,0.288194
4,1.805200,1.797462,0.295139
5,1.722100,1.660223,0.402778
6,1.577300,1.636248,0.447917
7,1.478500,1.467472,0.496528
8,1.382800,1.373512,0.500000
9,1.235200,1.288559,0.527778
10,1.159000,1.218379,0.552083


TrainOutput(global_step=720, training_loss=1.2880173795753056, metrics={'train_runtime': 4843.3573, 'train_samples_per_second': 4.757, 'train_steps_per_second': 0.149, 'total_flos': 1.39656982560768e+18, 'train_loss': 1.2880173795753056, 'epoch': 20.0})

In [34]:
trainer.evaluate()

{'eval_loss': 1.0280424356460571,
 'eval_accuracy': 0.6145833333333334,
 'eval_runtime': 16.3958,
 'eval_samples_per_second': 17.565,
 'eval_steps_per_second': 2.196,
 'epoch': 20.0}

In [35]:
trainer.push_to_hub()

Several commits (2) will be pushed upstream.
The progress bars may be unreliable.


Upload file runs/Aug18_14-04-44_01059d9d5083/events.out.tfevents.1692367833.01059d9d5083.2488.0:   0%|        …

Upload file runs/Aug18_14-04-44_01059d9d5083/events.out.tfevents.1692372692.01059d9d5083.2488.1:   0%|        …

To https://huggingface.co/Wiam/wav2vec2-large-robust-ft-libri-960h-finetuned-ravdess-v2
   127799a..936a763  main -> main

   127799a..936a763  main -> main

To https://huggingface.co/Wiam/wav2vec2-large-robust-ft-libri-960h-finetuned-ravdess-v2
   936a763..38001da  main -> main

   936a763..38001da  main -> main



'https://huggingface.co/Wiam/wav2vec2-large-robust-ft-libri-960h-finetuned-ravdess-v2/commit/936a76330b7b0c1f02c9701b7fbdd091bcae1182'